# EDA of Metro HubNashville Dataset

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
##import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from shapely.geometry import Point

%matplotlib inline

## Read in HubNashville data file

In [2]:
hub_data = pd.read_csv('../data/hubNashville__311__Service_Requests.csv', low_memory = False)

FileNotFoundError: [Errno 2] File ../data/hubNashville__311__Service_Requests.csv does not exist: '../data/hubNashville__311__Service_Requests.csv'

## My Questions
- what to do with nulls?
- fix date format?
- separate time from date column?


- longest open request?
- how quickly are most resolved?
- most common request type?
- what time do most requests come in?
- what month do most requests come in? 
- what council district has the most requests?    
- how many requests are made in a language other than English? No values in this column?  
- how many resolved the same day, in a week?
- how many escalated?
- What are the COVID-19 requests about?
- How has COVID-19 impacted (or will impact) public transportation? Animal control?

## Questions our team would like to answer:
- Trending requests: daily, weekly, monthly, yearly, time of day. Show most demanding requests trends daily, weekly, monthly... and show a comparison  over  the last 3 years, perhaps divide 24 hours into subsets. To possibly prepare for up coming request types and get an overview before digging deeper.
- Covid-19 impact (requests, violations) by council district
- Tornado impact - Hands On volunteers -- anything to improve for the next need?
- Parent requests - how many ‘child’ requests are added while the issue is unresolved.
- Average resolution time by type of request (this could be provided to people who make requests, so they have an ETA )


## Preliminary look at the data

In [ ]:
hub_data.head()

In [ ]:
hub_data.shape

In [ ]:
hub_data.info(verbose=True)

## Sort values by when request was opened

In [ ]:
hub_data.sort_values(by = 'Date / Time Opened')

## Looked at null values

In [ ]:
mapped_loc_nulls= hub_data['Mapped Location'].isnull()

In [ ]:
mapped_loc_nulls

In [ ]:
mapped_loc_nulls.value_counts()

## Convert to DateTime

In [ ]:
t1 = pd.to_datetime(hub_data['Date / Time Opened'])
t2 = pd.to_datetime(hub_data['Date / Time Closed'])
hub_data['time_taken'] = t1-t2
# move time_taken column to after time_closed
col_name = 'time_taken'
hub_time = hub_data.pop(col_name)
hub_data.insert(7, col_name, hub_time)


In [ ]:
hub_data['Date / Time Opened'] = pd.to_datetime(hub_data['Date / Time Opened'])

In [ ]:
hub_data['Date / Time Closed'] = pd.to_datetime(hub_data['Date / Time Closed'])

In [ ]:
hub_data.head(5)

## Create a year column

In [ ]:
##hub_data[['Year Requested']] = hub_data['Date / Time Opened'].str.split('-', n=1, expand=True)
##df['date'] = pd.to_datetime(df['date'])
##df['year'], df['month'] = df['date'].dt.year, df['date'].dt.month

hub_data['Year Requested'] = hub_data['Date / Time Opened'].dt.year

In [ ]:
hub_data.head()

## Look at counts in RequestType column

In [ ]:
hub_data['Request Type'].value_counts()

In [ ]:
hub_data['Subrequest Type'].value_counts()

## Look at counts in Status column

In [ ]:
hub_data['Status'].value_counts()

## Isolated COVID-19 requests

In [ ]:
covid_19_req = hub_data[hub_data['Request Type'] == 'COVID-19']
covid_19_req

## Subset dataframe with Subrequest for 'Dead Animal Pickup'

In [ ]:
animal_pickup = hub_data[(hub_data['Subrequest Type'].notnull())&(hub_data['Subrequest Type']=='Dead Animal Pickup')]

In [ ]:
animal_pickup.shape

In [ ]:
animal_pickup

## Subset dataframe with Subrequest for 'Debris'

In [ ]:
debris_sub = hub_data[(hub_data['Subrequest Type'].notnull())&(hub_data['Subrequest Type']=='Debris')]

In [ ]:
debris_sub.head()

## Map COVID-19 by zip code = requests, violations,  questions

### Create a geometry column

In [ ]:
hub_data['geometry'] = hub_data.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)


In [ ]:
hub_data_geo = gpd.GeoDataFrame(hub_data,  
                           geometry = hub_data['geometry'])
hub_data_geo

In [ ]:
print(hub_data_geo.crs) ## How do I set the crs?

## Bobcat 1
- Dead animal pickup and debris (for example) - make variable dataframes
- Count of dead animal pickup (or other categories), made histogram or other visual, count requests by zip 
- compare between different events (covid, tornado, control) and time of day. 
- Use this info to inform resource allocation.

## Bobcat 2
- Aggregate count of subrequest type by hour/month 
- How many of each (sub)request type are open at a given time to help staff forecast when heavy request load will occur.
- Consider dividing into 6hr chunks/bins for unit of analysis (morning, afternoon, evening, night)

## Bobcat 3
- Visualize geographical differences across time to completion/request type/other factors. 
- Identify spikes/outliers/trends across different dimensions (time/request type/volume/etc.). 
- Intention to help direct resources. Which area has the most activity to direct staff appropriately. 